# Setup: Multi-Terminal and Multi-Account Connection Test

This notebook demonstrates how to connect to multiple, simultaneously running MetaTrader 5 terminals and their respective accounts.

**Purpose:**

- To provide a clear example of managing connections to different brokers or accounts that are running in separate terminal instances.
- To test that the `MT5Connector` can handle different terminal paths and account sections sequentially.

**Prerequisites for running this notebook:**

1.  You must have two (or more) MT5 terminals installed.
2.  **Both terminals must be running at the same time.**
3.  The `config.ini` file must contain separate sections for each terminal path and each account.

**Workflow:**

1.  It connects to the first terminal (`terminal_alpha`) and logs into its corresponding account. It fetches the balance and then disconnects.
2.  It then connects to the second terminal (`terminal_beta`) and logs into its account, fetches the balance, and disconnects.
3.  Finally, it displays the data collected from both accounts.


In [ ]:
# --- 1. SETUP PHASE ---
import os
import sys

import MetaTrader5 as mt5

print("--- Initializing Setup ---")
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    config_file_path = os.path.join(project_root, "config.ini")

    if project_root not in sys.path:
        sys.path.append(project_root)

    from utils.mt5_connector import MT5Connector

    print("Setup successful. Connector is ready.")
except Exception as e:
    print(f"Error during setup: {e}")

In [ ]:
# --- 2. MULTI-CONNECTION PHASE ---

# Define the sections we will use from config.ini
TERMINAL_A_SECTION = "terminal_alpha"
ACCOUNT_A_SECTION = "account_alpha_demo"

TERMINAL_B_SECTION = "terminal_beta"
ACCOUNT_B_SECTION = "account_beta_demo"

# Variables to store the results from each connection
alpha_data = None
beta_data = None

# --- Connection to Terminal A ---
print(f"\n--- 1. Attempting to connect to terminal '{TERMINAL_A_SECTION}' ---")
try:
    with MT5Connector(
        config_path=config_file_path, terminal_section=TERMINAL_A_SECTION
    ) as conn_a:
        print(">>> Terminal A Initialized <<<")
        if conn_a.login_to_account(account_section=ACCOUNT_A_SECTION):
            print(">>> Account A Login Successful <<<")
            info = mt5.account_info()
            if info:
                alpha_data = {
                    "login": info.login,
                    "server": info.server,
                    "balance": info.balance,
                    "currency": info.currency,
                }
        else:
            print(">>> Account A Login Failed <<<")
except Exception as e:
    print(f"An error occurred with Terminal A: {e}")

# --- Connection to Terminal B ---
print(f"\n--- 2. Attempting to connect to terminal '{TERMINAL_B_SECTION}' ---")
try:
    with MT5Connector(
        config_path=config_file_path, terminal_section=TERMINAL_B_SECTION
    ) as conn_b:
        print(">>> Terminal B Initialized <<<")
        if conn_b.login_to_account(account_section=ACCOUNT_B_SECTION):
            print(">>> Account B Login Successful <<<")
            info = mt5.account_info()
            if info:
                beta_data = {
                    "login": info.login,
                    "server": info.server,
                    "balance": info.balance,
                    "currency": info.currency,
                }
        else:
            print(">>> Account B Login Failed <<<")
except Exception as e:
    print(f"An error occurred with Terminal B: {e}")


# --- 3. DISPLAY RESULTS ---
print("\n" + "=" * 40)
print("--- MULTI-CONNECTION TEST SUMMARY ---")
print("=" * 40)

if alpha_data:
    print(f"\nData from Terminal A (Account #{alpha_data['login']}):")
    print(f"  - Server: {alpha_data['server']}")
    print(f"  - Balance: {alpha_data['balance']} {alpha_data['currency']}")
else:
    print("\nCould not retrieve data from Terminal A.")

if beta_data:
    print(f"\nData from Terminal B (Account #{beta_data['login']}):")
    print(f"  - Server: {beta_data['server']}")
    print(f"  - Balance: {beta_data['balance']} {beta_data['currency']}")
else:
    print("\nCould not retrieve data from Terminal B.")

print("\nTest finished.")